# Collaboration and Competetion

---

*Jun Zhu, zhujun981661@gmail.com, 09.2020*

In this notebook, I present the solution for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

# !pip install matplotlib

import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')
from utilities import check_environment, play, plot_score_history, plot_losses

Next, we will start the environment! Before running the code cell below, change the file_name parameter to match the location of the Unity environment that you downloaded.

    Mac: "path/to/Tennis.app"
    Windows (x86): "path/to/Tennis_Windows_x86/Tennis.exe"
    Windows (x86_64): "path/to/Tennis_Windows_x86_64/Tennis.exe"
    Linux (x86): "path/to/Tennis_Linux/Tennis.x86"
    Linux (x86_64): "path/to/Tennis_Linux/Tennis.x86_64"
    Linux (x86, headless): "path/to/Tennis_Linux_NoVis/Tennis.x86"
    Linux (x86_64, headless): "path/to/Tennis_Linux_NoVis/Tennis.x86_64"

For instance, if you are using a Mac, then you downloaded Tennis.app. If this file is in the same folder as the notebook, then the line below should appear as follows:

env = UnityEnvironment(file_name="Tennis.app")

In [ ]:
env = UnityEnvironment(file_name='Tennis_Linux/Tennis.x86_64')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1. If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01. Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping.

Run the code cell below to print some information about the environment.

In [ ]:
brain_name, state_space, action_space = check_environment(env)
brain_name

### 3. Take Random Actions in the Environment

In [ ]:
play(env, brain_name, continuous=True)